In [1]:
import numpy as np
import os
import pandas as pd
from datetime import datetime as dt

In [2]:
DATA_PATH='../../SCHOOL/INFO633/ASSIN 3/data/'

---
### CENCUS DATASET

In [3]:
cencus_data='nc-est2019-sr11h.xlsx'

pop_data_file=pd.read_excel(DATA_PATH+cencus_data, skiprows=3)

In [14]:
POPULATION_DATA=pd.DataFrame()
POPULATION_DATA['Cases_Total']=[pop_data_file.T[0].loc[2019], (pop_data_file.T[0].loc[2019])]
POPULATION_DATA['Cases_White']=197304822#[pop_data_file.T[2].loc[2019], (pop_data_file.T[2].loc[2019])]
POPULATION_DATA['Cases_Black']=[pop_data_file.T[3].loc[2019], (pop_data_file.T[3].loc[2019])]
POPULATION_DATA['Cases_LatinX']=[pop_data_file.T[28].loc[2019], (pop_data_file.T[28].loc[2019])]
POPULATION_DATA['Cases_Asian']=[pop_data_file.T[5].loc[2019],(pop_data_file.T[5].loc[2019])]
POPULATION_DATA['Cases_AIAN']=[pop_data_file.T[4].loc[2019],(pop_data_file.T[4].loc[2019])]
POPULATION_DATA['Cases_NHPI']=[pop_data_file.T[6].loc[2019], (pop_data_file.T[6].loc[2019])]
POPULATION_DATA['Cases_Multiracial']=[pop_data_file.T[7].loc[2019], (pop_data_file.T[7].loc[2019])]
POPULATION_DATA.iloc[1]=POPULATION_DATA.iloc[0]/POPULATION_DATA['Cases_Total'][0]*100

In [15]:
POPULATION_DATA.to_excel(DATA_PATH+'population_df.xlsx', index=False)

---
### ORIGINAL_DATASET Cleaning

In [24]:
COVID_DATA=pd.read_csv(DATA_PATH+'Race Data Entry - CRDT (1).csv', parse_dates=['Date'])

In [48]:
COVID_DATA=COVID_DATA[~COVID_DATA['State'].isin(['AS','GU','MP','PR','VI'])]


In [12]:
for cases_i in COVID_DATA.columns[2:]:
    new_case=[]
    for num, i in enumerate(COVID_DATA[cases_i]):
        if type(i) is str:
            i=(float(i.replace(',', '')))
        new_case.append(i)
    COVID_DATA[cases_i]=new_case

In [13]:
COVID_DATA[COVID_DATA.columns[2:]]=COVID_DATA[COVID_DATA.columns[2:]].astype(float)

----
#### DATASET OF SUMMATION OF CASES/DEATH BY DATE

In [14]:
SUM_DATASET=pd.DataFrame({'Date':np.unique(COVID_DATA['Date'])})
DIFF_SUM_DATASET=pd.DataFrame({'Date':np.unique(COVID_DATA['Date'])})
PERCENT_DATASET=pd.DataFrame({'Date':np.unique(COVID_DATA['Date'])})


for i in COVID_DATA[COVID_DATA.columns[2:]]:
    SUM_DATASET[i]=np.nan
    DIFF_SUM_DATASET[i]=np.nan

In [15]:
for case_death_i in COVID_DATA[COVID_DATA.columns[2:]]:
    for date_i in np.unique(COVID_DATA['Date']):
        sumed=np.sum(COVID_DATA[case_death_i][COVID_DATA['Date']==date_i])
        SUM_DATASET.loc[SUM_DATASET.Date==date_i, case_death_i]=sumed

#### ADDING DAILY CHANGE COLUMNS

In [16]:
for sum_col, diff_col in zip(SUM_DATASET.columns[1:],DIFF_SUM_DATASET.columns[1:]):
    diff=[0] 
    for i, sum_i in enumerate(SUM_DATASET[sum_col][1:]):
        daily_change=sum_i-SUM_DATASET[sum_col][i]
        diff.append(daily_change)
    DIFF_SUM_DATASET[diff_col]=diff

#### ADDING % COLUMNS

In [27]:
total_case_sum=SUM_DATASET['Cases_Total']-SUM_DATASET['Cases_Unknown']
total_death_sum=SUM_DATASET['Deaths_Total']-SUM_DATASET['Deaths_Unknown']

## CASE COLUMNS
for sum_col in SUM_DATASET.columns[2:10]:
    tar_col=SUM_DATASET[sum_col]
    perc_col=(tar_col/total_case_sum)*100
    PERCENT_DATASET[sum_col]=perc_col
    
## DEATH COLUMNS
for sum_col in SUM_DATASET.columns[14:24]:
    tar_col=SUM_DATASET[sum_col]
    perc_col=(tar_col/total_death_sum)*100
    PERCENT_DATASET[sum_col]=perc_col

In [35]:
POPULATION_SCALED_DF=pd.DataFrame({'Date':np.unique(SUM_DATASET['Date'])})
for percent, pop_cent in zip(SUM_DATASET[SUM_DATASET.columns[1:9]],POPULATION_DATA[POPULATION_DATA.columns[:]]):
    POPULATION_SCALED_DF[percent]=SUM_DATASET[percent]/POPULATION_DATA[pop_cent].iloc[0]*100
for percent, pop_cent in zip(SUM_DATASET[SUM_DATASET.columns[14:22]],POPULATION_DATA[POPULATION_DATA.columns[:]]):
    POPULATION_SCALED_DF[percent]=SUM_DATASET[percent]/POPULATION_DATA[pop_cent].iloc[0]*100

---
#### SAVING:

In [31]:
# SUM_DATASET.to_excel(DATA_PATH+'DAILY_SUMS.xlsx', index=False)
# COVID_DATA.to_excel(DATA_PATH+'OLD_CLEAN_DATASET.xlsx', index=False)
# PERCENT_DATASET.to_excel(DATA_PATH+'PERCENT_DATA.xlsx', index=False)
# DIFF_SUM_DATASET.to_excel(DATA_PATH+'DIFF_SUM_DATA.xlsx', index=False)